<a href="https://colab.research.google.com/github/mahendra-gehlot/CP260/blob/main/CP260_Lab_1_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lab 1: Discrete Bayes Filter**
In this lab, you will implement a discrete bayes filter
to track the 1D motion of a robot Ant. The noise in the ant's motion and its sensor have a triangular probability mass function (PMF). You can control the ant's velocity, its initial position, the spread of its noise. In addition the ant's velocity changes to a new random value of after runLength number of time steps. This will allow you to test if your filter is able to track the ant's position when its velocity is different from what you assume.

You have to write two equations, one for predict and another for update for the discrete bayes filter. You can use embedded latex to work out the syntax.
https://colab.research.google.com/github/bebi103a/bebi103a.github.io/blob/master/lessons/00/intro_to_latex.ipynb#scrollTo=Lf4u9FLje0Gd
has a brief tutorial on this.

You have to also write python code for those two functions also.
Here is one such tutorials. There are plenty more.
https://colab.research.google.com/github/cs231n/cs231n.github.io/blob/master/python-colab.ipynb

Finally, you need to calculate tabulate the rmserror for different
conditions and embed the result in the final panel.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as ip


**Discrete Bayes Filter**

Assume a robotic ant which moves in a circle with positions 0 to N-1

State of the robot is: $ x_t \in [0,N-1] $

Process Model (aka Motion Model): 
$x_{t+1} = (x_t + u_t  + r) \% N $

$u_t$ is the change in position per unit time (velocity),
and $r$ is the random noise with probability mass over $[-k, k]$.

For this exercise, we will assume a triangular PMF with
$P(r=i) = \frac{k+1-|i|}{(k+1)^2} $

Measurement model is: $z_t = (x_t + q ) \% N $
where $q$ is a random noise with probability mass over $[-l,l]$

with a triangular PMF with
$P(q=i) = \frac{l+1-|i|}{(l+1)^2}$

By adjusting $k,m$ we can adjust the noise process' variance.

In [ ]:
# Dont edit this panel
class SimAnt:
  '''Simulate a stochastic Ant walking around in a circle'''
  def __init__(self, 
                initPos = 0, #initial position of dog 
                initVel=1, #initial velocity of dog
                minVel = -3, #minimum velocity
                maxVel = 3, #maximum velocity
                runLength = 25, # number of steps with const. velocity
                numBins=10, #lenght of circle around which the ant is walking
                sensorNoiseLimit = 3, # The sensor adds noise which is +- senseNoiseLimit of current pos
                motionNoiseLimit = 3, # The noise adds to motion which is +- motionNoiseLimit of new pos
                randWalk=False, #if set as false, the dog's motion is not random 
                randSeed=0 #allows reproduction of the motion across runs
                ):
    self.randWalk=randWalk
    self.time = 0. #current time
    self.baseTime = 0 #base time for measuring run length to allow switch of velocity
    self.rngMotion = np.random.default_rng(seed=randSeed) #random gen for motion
    self.rngMeas = np.random.default_rng(seed=randSeed) #random gen for measurement
    self.pos = initPos #initial position
    self.numBins = numBins #number of slots in the circle, numbered 0..N-1
    self.vel = initVel
    self.minVel = minVel
    self.maxVel = maxVel
    self.runLength = runLength
    l=sensorNoiseLimit
    self.sensorNoise = np.arange(-l, l+1,1) #list of possible sensor noise values
    self.sensorNoisePMF = [(l+1-abs(i))/(l+1)**2 for i in self.sensorNoise]
    m=motionNoiseLimit
    self.motionNoise = np.arange(-m, m+1,1) #list of possible motion noise values
    self.motionNoisePMF = [(m+1-abs(i))/(m+1)**2 for i in self.motionNoise]
    print("Created Ant at time=", self.time, "with pos=",self.pos, " velocity=",self.vel)
    print("sensorNoise=",self.sensorNoise, "motionNoise=",self.motionNoise)
    print("sensorNoisePMF=",self.sensorNoisePMF, "motionNoisePMF=",self.motionNoisePMF)

  def step(self):
    #implement motion of the dog
    self.time = self.time + 1
    if (self.randWalk):
      if (self.time == self.baseTime + self.runLength):
        #time to randomly change velocity
        self.vel = self.rngMotion.integers(low=self.minVel, high=self.maxVel)
        self.baseTime = self.time
    #generate motion noise as per the motionNoisePMF
    noise = self.rngMotion.choice(self.motionNoise,p=self.motionNoisePMF)
    self.pos = (self.pos + self.vel + noise) % self.numBins


  def getMeasuredPos (self):
    '''return position with measurement noise added'''
    noise = self.rngMeas.choice(self.sensorNoise,p=self.sensorNoisePMF)
    return (self.pos + noise)%self.numBins

  def getTruePos (self):
    return(self.pos)
  
  def getTime (self):
    return(self.time)


Question 1: Write the equation of the Prediction of belief State below.



Your answer here:

$
bel(\bar{x_t}=k) =  \begin{gather*}
    \sum P(x_t = k | u_t , x_{t-1} = j) bel(x_{t-1} = j) \end{gather*}
$


Question 2: Write the equation for the update given the measurement



Your Answer Here:

$bel(x_t=k)= \eta P(z_t | x_t = k) \overline {bel} (x_t = k ) 
$

In [ ]:
  #Edit two function predict and update
class DiscreteBayesFilter1D:
  ''' Discrete Bayes Filter of 1 dimension with numBins discrete steps per dimension'''
  def __init__(
    self,
    numBins = 10, #number of discrete bins per dimension   
    motionNoiseLimit = 2, #motion noise limit. Assume triangular PMF for +- motionNoiseLimit   
    sensorNoiseLimit = 2, #sensor noise limit. Assume triangular PMF for +- sensorNoiseLimit   
    initPos = 0, #initial position
    initSpread = 0, #Uniform probability in initial prior between initPos += initSpread
  ):
    self.n = numBins
    self.initPos = initPos
    self.initSpread = initSpread
    self.reset() #initialize the beliefs (see below for function)
    m = motionNoiseLimit
    self.m = m
    self.motionNoise = np.arange(-m, m+1,1)
    self.motionNoisePMF = [(m+1-abs(i))/(m+1)**2 for i in range(-m,m+1)] #triangular PMF for motion noise
    l=sensorNoiseLimit
    self.l = l
    self.sensorNoise = np.arange(-l, l+1,1) #list of possible sensor noise values
    self.sensorNoisePMF = [(l+1-abs(i))/(l+1)**2 for i in self.sensorNoise]
    print ("created discrete bayes filter with sensor noise=", self.sensorNoise,"and PMF=",self.sensorNoisePMF)
    
  def predict(self,u_t):
    Post_Mat = []
    for k in range(self.n):
      Row = []
      for j in range(self.n):
        index = []
        flag = True
        for N in self.motionNoise:
          check = j+ u_t + N
          if check < 0:
            check = check % self.n;
            check = check + self.n;
          if check > self.n-1:
            check = check % self.n
          if (check) == k:
            index = np.where(self.motionNoise == N)
            index = index[0][0]
            Row.append(self.motionNoisePMF[index])
            flag = False
            break
        if flag:
          Row.append(0)  
      Post_Mat.append(Row)
    Post_Mat = np.array(Post_Mat)
    self.belPrior = np.matmul(Post_Mat,self.bel)
  

  def update (self,z):
    '''update the posterior based on the measurement and predicted prior'''
    row = []
    for i in range(self.n):
      for q in self.sensorNoise:
        flag = True
        check = z+q
        if check > (self.n -1):
          check = check % self.n;
        if check < 0:
          check = check % self.n
          check = check + self.n
        if (check) == i:
          index = np.where(self.sensorNoise == q)
          index = index[0][0]
          row.append(self.sensorNoisePMF[index])
          flag = False
          break
      if flag:
        row.append(0)
    new = np.multiply(row,self.belPrior)
    neta = np.sum(new)
    self.bel = new/neta
        
    

  def getPos (self):
    '''get the mode of the belief as the estimate of position'''
    avgIdx = 0
    for i in range(self.bel.shape[0]):
      avgIdx += i * self.bel[i]
    return(avgIdx)
  
  def reset (self):
    '''Reset the state of the filter'''
    #print("----CALLING dbf.reset---------")
    self.belPrior = np.zeros(self.n) #initialize the belief prior array
    self.bel = np.zeros(self.n) #initialize the belief posterior array
    for i in range(-self.initSpread+self.initPos, self.initSpread+self.initPos+1):
      self.bel[i] = 1/(2*self.initSpread+1) #initalize the start belief to be uniform.
  
  def getBelPrior (self):
    return (self.belPrior)
  
  def getBel (self):
    return (self.bel)


In [ ]:
ant = SimAnt(initPos=0,sensorNoiseLimit=3, motionNoiseLimit = 3, numBins=10,randWalk=False)
dbf = DiscreteBayesFilter1D(numBins=4, sensorNoiseLimit=1, motionNoiseLimit=1, initPos=0, initSpread=0)


Created Ant at time= 0.0 with pos= 0  velocity= 1
sensorNoise= [-3 -2 -1  0  1  2  3] motionNoise= [-3 -2 -1  0  1  2  3]
sensorNoisePMF= [0.0625, 0.125, 0.1875, 0.25, 0.1875, 0.125, 0.0625] motionNoisePMF= [0.0625, 0.125, 0.1875, 0.25, 0.1875, 0.125, 0.0625]
created discrete bayes filter with sensor noise= [-1  0  1] and PMF= [0.25, 0.5, 0.25]


In [ ]:
dbf.reset()
dbf.predict(0)
dbf.update(1)
dbf.getBel()
dbf.reset()

In [ ]:
#unit testing
dbf.reset()
print("belPrior=",dbf.getBelPrior())
print("bel = ",dbf.getBel())
dbf.predict(1)
print("belPrior=",dbf.getBelPrior())
dbf.update(3)
print("bel=",dbf.getBel())
dbf.reset()

belPrior= [0. 0. 0. 0.]
bel =  [1. 0. 0. 0.]
belPrior= [0.25 0.5  0.25 0.  ]
bel= [0.5 0.  0.5 0. ]


In [ ]:
# test the ant motion
for t in np.arange(2):
  ant.step();
  dbf.predict(1)
  dbf.update(ant.getMeasuredPos())
  print("time=", ant.getTime(), " truePos=",ant.getTruePos()," measuredPos=", ant.getMeasuredPos(), "estimated Pos=",dbf.getPos())
  print("bel=",dbf.getBel())
ant.__init__()
dbf.__init__()

time= 1.0  truePos= 2  measuredPos= 1 estimated Pos= 1.0
bel= [0.5 0.  0.5 0. ]
time= 2.0  truePos= 2  measuredPos= 9 estimated Pos= 1.0
bel= [0.25 0.5  0.25 0.  ]
Created Ant at time= 0.0 with pos= 0  velocity= 1
sensorNoise= [-3 -2 -1  0  1  2  3] motionNoise= [-3 -2 -1  0  1  2  3]
sensorNoisePMF= [0.0625, 0.125, 0.1875, 0.25, 0.1875, 0.125, 0.0625] motionNoisePMF= [0.0625, 0.125, 0.1875, 0.25, 0.1875, 0.125, 0.0625]
created discrete bayes filter with sensor noise= [-2 -1  0  1  2] and PMF= [0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.2222222222222222, 0.1111111111111111]


In [ ]:
#simulates the ant and runs the discrete bayes filter
#rmserror in position tracking is recorded
def simulate (numTimeSteps=10,numBins=10,
              #ant simulation parameters
              antInitVel=1, antInitPos=0, 
              antSensorNoiseLimit=1, antMotionNoiseLimit=1, antRunLength=25,
              randWalk=True,
              #predictor parameters
              predInitPos=0,predInitSpread=0, predVel=1,
              predSensorNoiseLimit = 1, predMotionNoiseLimit=1
              ):
  ant.__init__(numBins=numBins,initPos=antInitPos,sensorNoiseLimit=antSensorNoiseLimit,
               motionNoiseLimit=antMotionNoiseLimit,initVel=antInitVel,runLength=antRunLength,randWalk=randWalk)
  dbf.__init__(numBins=numBins, sensorNoiseLimit=predSensorNoiseLimit,motionNoiseLimit=predMotionNoiseLimit,
               initPos=predInitPos, initSpread=predInitSpread)
  time = np.zeros(numTimeSteps)
  antTruePos = np.zeros(numTimeSteps)
  antMeasuredPos = np.zeros(numTimeSteps)
  antEstimatedPos = np.zeros(numTimeSteps)
  rmsError = 0
  for i in range(numTimeSteps):
    ant.step()
    time[i]=i
    antTruePos[i]=ant.getTruePos()
    mPos = ant.getMeasuredPos()
    antMeasuredPos[i]=mPos
    dbf.predict(predVel)
    dbf.update(mPos)
    antEstimatedPos[i] = dbf.getPos()
    rmsError += (antEstimatedPos[i]-antTruePos[i])**2
  print(numTimeSteps)
  rmsError /= numTimeSteps
  rmsError = rmsError**0.5
  print("rms position error = ", rmsError)

  #print("time=",time, "antTruePos=",antTruePos)
  #plot the values
  plt.figure(figsize=(5,5))
  plt.plot(time,antTruePos, label='antTruePos')
  plt.plot(time,antMeasuredPos,label='antMeasuredPos')
  plt.plot(time,antEstimatedPos,label='antEstimatedPos')
  plt.xlabel('timestep')
  plt.ylabel('pos')
  plt.title("ant 1D motion simulation")
  plt.legend()
  


In [ ]:
#Adjust the slider bar to get results for various scenarious
ip.interact(simulate,
            numTimeSteps=(50,200,50),         #number of timesteps for this simulation
            numBins = (10,50,5),              #number of bins in the race course (it is circular)
            antInitVel = (-4,4,1),            #ant's initial velocity
            antInitPos = (-4,4,1),            #ant's initial position
            antSensorNoiseLimit = (0,5,1),    #ant's sensor noise
            antMotionNoiseLimit = (0,5,1),    #ant's motion noise
            predSensorNoiseLimit = (0,5,1),   #estimator's model of sensor noise. Triangular Pmf from -limit to +limit
            predMotionNoiseLimit = (0,5,1),   #estimator's model of motion noise. Triangular Pmf from -limit to +limit
            predVel = (-4,4,1),               #velocity used by the estimator in its motion model
            predInitPos = (-4,4,1),           #inital pos used by the estimator
            predInitSpread = (0,5,1),         #spread in position uncertainty (initPos +- initSpread)
            antRunLength = (100,200,50),      #number of consecutive simulation steps over which ant's vel is const. after that a  
            )

interactive(children=(IntSlider(value=50, description='numTimeSteps', max=200, min=50, step=50), IntSlider(val…

<function __main__.simulate>

Fill in the rms pos error values for the settings indicated
in the table below. Replace '?' with your observed value

\begin{array}{ |c|c|c|c|c|c||c| }
\hline
antSensorNoiseLimit & antMotionNoiseLimit & predInitPos & predVel &predSensorNoiseLimit & predMotionNoiseLimit & rmsPosError\\
\hline
0 & 0 & 0 & 1 & 0 & 0 & 0\\
\hline
1 & 1 & 0 & 1 & 1 & 1 & 2.73\\
\hline
1 & 1 & 0 & 1 & 3 & 1 & 2.83\\
\hline
1 & 1 & 0 & 1 & 1 & 3 & 2.73\\
\hline
1 & 1 & 0 & 1 & 3 & 3 & 2.83\\
\hline
1 & 1 & 0 & 2 & 1 & 1 & 2.98\\
\hline
\end{array}


$ Brief Discussion $

Your brief comments on the results above here.

MGE: The Bayes filter is framework for state estimation. FurtherMore, From above exercise, We can interpret that high noise in any parameter will result in high RMS error. Hence accurate sensor will result in better estimation of state.


